In [ ]:
import pandas as pd
import geopandas as gpd
import random
import fiona
from shapely.geometry import Point
from geopy.distance import geodesic
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt
from pathlib import Path
import random, os, sys
import numpy as np
import math
import warnings; warnings.simplefilter('ignore')
import seaborn as sns

sys.path.insert(0, 'C:/Users/mmh/Documents/Codes/cross-sectro-transp-energy-model/')
# importlib.reload(sys.modules['src.calculation'])
from src.func_buildings import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data_dir = 'C:/Users/mmh/Documents/Data/'
data_dir2 = 'C:/Users/mmh/OneDrive - Oak Ridge National Laboratory/Melrose/9.Data/'
map_dir = 'C:/Users/mmh/OneDrive - Oak Ridge National Laboratory/Melrose/8.Maps/'

# %store -r building_cap
# %store -r building_loc

In [ ]:
shp = gpd.read_file(map_dir + 'geo/tl_2021/tl_2021_13_bg.zip') # Georgia
# shp = shp[(shp.STATEFP == '13')&(shp.COUNTYFP == '059')]
shp = shp[(shp.STATEFP == '13')&(shp.COUNTYFP == '121')] # Fulton
print(len(shp))
shp.head(2)

In [ ]:
zones = shp.loc[(shp.STATEFP == '13')&(shp.COUNTYFP == '059'),'GEOID'].values
od_application = od_long[(od_long.COUNTY_origin=='13059')&(od_long.COUNTY_dest=='13059')]  
print(len(set(od_application.origin)))
od_application.head()

# Trip chain generation

In [ ]:
def generate_trip_chain(init_o,num_veh,trajectory=[]):
    for veh in range(1,num_veh+1):# Loop through all vehicles that start from this zone
        print('vehicle id: '+str(veh))
        #### step 2: number of trips in the trip chain
        ntrips = 0
        while ntrips <= 0: #select one value based on normal distribution
            ntrips = int(np.random.normal(loc=ntrips_mean, scale=ntrips_std, size=1))
        trip_id = 1
        print('step 2 output: '+str(ntrips)+' trips in trip chain')
        
        while trip_id <= ntrips: 
            #### step 3: departure time (30 min granularity)          
            if trip_id == 1:
                start_time = np.random.choice(nhts_start_all[ntrips,trip_id].start_time, 1, 
                                            p=nhts_start_all[ntrips,trip_id].share)[0]
                start_hour = int(start_time) 
                init_start_hour = start_hour
                # Filter the OD data for the current hour
                od_hour_df = od_application[od_application['hour'] == start_hour] # od application should be an input??
                od_hour_df = od_hour_df[od_hour_df.origin == init_o] # initial starting point
                start_zone = init_o
            else:         
                start_hour = int(departure_time) 
                start_time = departure_time                
                # Filter the OD data for the current hour
                od_hour_df = od_long[od_long['hour'] == start_hour]
                od_hour_df = od_hour_df[od_hour_df.origin == next_zone] 
                start_zone = next_zone
            print('step 3 output: '+str(start_time)+' departure. Start zone: '+str(start_zone))
        
            #### step 4: trip purpose
            purpose_origin = np.random.choice(nhts_purpose_origin[ntrips,trip_id].loc[nhts_purpose_origin[ntrips,trip_id].start_time==start_time,
                                                                                      'origin_purpose'], 1, 
                                p=nhts_purpose_origin[ntrips,trip_id].loc[nhts_purpose_origin[ntrips,trip_id].start_time==start_time,'share'])[0]   
            purpose = np.random.choice(nhts_purpose_all[ntrips,trip_id].loc[nhts_purpose_all[ntrips,trip_id].start_time==start_time,
                                                                            'purpose'], 1, 
                                p=nhts_purpose_all[ntrips,trip_id].loc[nhts_purpose_all[ntrips,trip_id].start_time==start_time,'share'])[0]   
            print('step 4 output: '+str(purpose)+' trip purpose')
                
            #### step 5: possible destinations         
            possible_zones = od_hour_df.copy()
            
            # Generate next zone
            possible_zones['prob'] = possible_zones.trips/possible_zones.trips.sum()
            prob = possible_zones['prob'].ravel()
                   
            next_zone = np.random.choice(possible_zones.destination.values, p=prob)
            driving_dist = possible_zones.loc[(possible_zones.destination==next_zone),'dist'].values[0]         
            avg_speed = 40 # miles per hour
            driving_time = driving_dist/avg_speed
            arrival_time = start_time + driving_time
            print('step 6 output: '+str(next_zone)+' next zone')

            #### step 7: determine next departure time and stay time
            trip_id = trip_id + 1
            if trip_id <= ntrips and arrival_time < 20 : # heuristic
                departure_time = np.random.choice(nhts_start_all[ntrips,trip_id].start_time, 1, 
                                            p=nhts_start_all[ntrips,trip_id].share)[0]    
                if departure_time <= arrival_time:
                    departure_time = math.ceil(arrival_time * 2) / 2 
                stay_time = departure_time - arrival_time # occupancy
            else:
                stay_time = 24-arrival_time + init_start_hour
                if trip_id <= ntrips:
                    ntrips = trip_id-1
            print('step 7 output: '+str(stay_time)+' hours stay time')
            
            # save trajectory:
            trip_info = [trip_id-1,veh,start_time,start_zone,purpose_origin,purpose,arrival_time,next_zone,driving_dist,avg_speed,stay_time]
            trajectory.append(trip_info)

    trajectory_df = pd.DataFrame(trajectory, columns=['trip_id','veh_id', 'start_time', 'start_zone', 'purpose_origin', 'purpose', 'arrival_time', 'end_zone','distance','avg_speed','stay_time'])
    
    # assign to building
    trajectory_df = bg_to_building(trajectory_df)
    return(trajectory_df)

# Application in Clark County to estiamte charging load

In [ ]:
# determine base population based on census
nhts_pov = nhts[nhts['TRPTRANS'].isin([3,4,5,6,8,9,18])] # POV
nhts_pov = nhts_pov[nhts_pov['rank'] == 1]

# bg pop
zone_population = pd.read_csv(data_dir+'Census/ACSDT5Y2019.B01001 GA BG/ACSDT5Y2019.B01001-Data.csv')
zone_population = zone_population.loc[1:,['GEO_ID','B01001_001E']]
zone_population['GEO_ID'] = zone_population['GEO_ID'].str[9:]
zone_population.columns = ['origin','pop']

zone_population['COUNTY_origin'] = zone_population['origin'].str[:5]
zone_population['pop'] = pd.to_numeric(zone_population['pop'])
zone_population['origin'] = pd.to_numeric(zone_population['origin'])
print(len(zone_population))
zone_population.head()

# county pop
county_population = zone_population.groupby('COUNTY_origin').agg({'pop':'sum'}).reset_index()
county_population.head()

# calculate veh
nhts_total = nhts_pov['WTTRDFIN'].sum()/365
nhts_total_059 = nhts_total * county_population.loc[county_population.COUNTY_origin=='13059','pop'].values[0]/county_population['pop'].sum()
bg_base = zone_population[zone_population.COUNTY_origin=='13059']
print(len(bg_base))
bg_base['base_veh'] = bg_base['pop']/bg_base['pop'].sum() * nhts_total_059
bg_base.head()

In [ ]:
# zones = shp.loc[(shp.STATEFP == '13')&(shp.COUNTYFP == '059'),'GEOID'].values
# np.where(zones == '130590302003')[0]

In [ ]:
# single processing
zones = shp.loc[(shp.STATEFP == '13')&(shp.COUNTYFP == '121'),'GEOID'].values
# zones = [130591406001]

for i in range(len(zones)): # len(zones)
    init_o = int(zones[i])
    if os.path.isfile(os.path.join(data_dir, 'OD/Fulton County_v2/', 'trip_chain_' + str(init_o) + '.feather')):
        continue
    num_veh = 100
    trajectory_df = generate_trip_chain(init_o,num_veh,trajectory=[])
    trajectory_df.to_feather(data_dir+'OD/Fulton County_v2/trip_chain_'+str(init_o)+'.feather')
    # v1: Buildings with high occ are removed
    # v2: Keep all buildings

In [ ]:
np.where(zones == '131210082041')[0]

In [ ]:
# v2
zones = shp.loc[(shp.STATEFP == '13')&(shp.COUNTYFP == '059'),'GEOID'].values
# zones = [130591406001]
for i in range(len(zones)): # 
    init_o = int(zones[i])
    # base_veh = bg_base.loc[bg_base.origin==init_o,'base_veh']
    # if len(base_veh)==0:
    #     continue
    # num_veh = int(bg_base.loc[bg_base.origin==init_o,'base_veh'].values[0])
    # print(num_veh)
    num_veh = 100
    trajectory_df = generate_trip_chain(init_o,num_veh,trajectory=[])
    # trajectory_df['init_o'] = i
    trajectory_df.to_feather(data_dir+'OD/Clarke County_v3/trip_chain_'+str(init_o)+'.feather')